## Prompts for locations

In [1]:
location_prompts = [
    "steampunk city with skyscrapers",
    # "cyberpunk village in Japanese rustic style",
    # "fantasy dungsseons and dragons",
    # "noir city from 1930s",
    # "StarTrek inspired spaceship",
    # "undeground mine of goblins",
    # "SuperMario style magic land plain",
    # "SuperMario style magic land beach",
]

## Generating locations 

In [2]:
from holodeck import initialize_location
from holodeck import generate_location_and_encounters
import os
from tqdm.notebook import tqdm
import traceback

import concurrent.futures
from tqdm import tqdm

def generate_location(prompt):
    location_dict, encounters_list = generate_location_and_encounters(prompt)
    if location_dict:
        try:
            location = initialize_location(location_dict, encounters_list)
            return location
        except Exception as e:
            print("Error: ", e)
            traceback.print_exc()
    else:
        print(f"GENERATING FROM '{prompt}' failed!")
        return None

locations = []
with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    results = list(tqdm(executor.map(generate_location, location_prompts), total=len(location_prompts), desc="Generating locations"))
    locations += [r for r in results if r is not None]


assert len(locations) == len(location_prompts), "some locations are failed to generate"

locations

Generating locations:   0%|          | 0/1 [00:00<?, ?it/s]

<
{'name': 'The City of Avarice', 'description': 'A towering metropolis with steampunk inspired buildings and ornate steam powered automobiles filling the air with soot. The city is filled with skyscrapers that reach up to the sky, interspersed with large clock towers and old cathedrals. The streets are cobblestone lined, and the buildings are all built of an obsidian-like substance.', 'buildings': [{'name': 'Clock Tower', 'description': 'A tall, ornate clock tower with intricate carvings and an array of gears and cogs powering its clock face.', 'enterable': False}, {'name': 'Obsidian Building', 'description': 'A tall, obsidian building with a large door at its base and no windows.', 'enterable': True}, {'name': 'Cathedral of Avarice', 'description': 'A grand cathedral with grand stained glass windows and a large, ornate altar.', 'enterable': True}], 'ways': [{'name': 'Main Street', 'description': 'The main thoroughfare of the city, lined with steampunk inspired buildings and shops.'}]

Generating locations: 100%|██████████| 1/1 [00:14<00:00, 14.37s/it]

<
[{'probability': 0.4, 'description': 'You find an old man struggling to open the door of an obsidian building', 'actions': [{'type': 'character', 'name': 'Old man', 'description': 'An old man struggling to open the door of an obsidian building.'}], 'trigger': {'type': 'building', 'building': 'Obsidian Building'}}, {'probability': 0.2, 'description': 'You hear a loud crash coming from the cathedral', 'actions': [{'type': 'character', 'name': 'Robber', 'description': 'A robber, who has broken into the cathedral and is making off with a valuable artefact.'}], 'trigger': {'type': 'building', 'building': 'Cathedral of Avarice'}}, {'probability': 0.1, 'description': 'You hear a loud ticking from the clock tower', 'actions': [{'type': 'critter', 'name': 'Mechanical rat', 'description': 'A mechanical rat, with gears and cogs instead of fur and skin, scurrying through th'}]}]



[Location(name='The City of Avarice', description='A towering metropolis with steampunk inspired buildings and ornate steam powered automobiles filling the air with soot. The city is filled with skyscrapers that reach up to the sky, interspersed with large clock towers and old cathedrals. The streets are cobblestone lined, and the buildings are all built of an obsidian-like substance.', buildings=[Building(name='Clock Tower', description='A tall, ornate clock tower with intricate carvings and an array of gears and cogs powering its clock face.', enterable=False), Building(name='Obsidian Building', description='A tall, obsidian building with a large door at its base and no windows.', enterable=True), Building(name='Cathedral of Avarice', description='A grand cathedral with grand stained glass windows and a large, ornate altar.', enterable=True)], ways=[Way(name='Main Street', description='The main thoroughfare of the city, lined with steampunk inspired buildings and shops.')], encounter

In [3]:
locations[0]\
    .encounters[1]\
        .triggers[0]

Trigger(type=<TriggerType.BUILDING: 'building'>, way=None, building=Building(name='Central Square', description='The heart of Skyport. Everything converges here, and the airship docks are located in this area.', enterable=True))

In [4]:
for location in locations:
    for encounter in location.encounters:
        for trigger in encounter.triggers:
            print(trigger)



Trigger(type=WAY, way=Main Road, encounter_id=None)
Trigger(type=WAY, way=River Riveria, encounter_id=None)


## Save Locations to sqlite

In [3]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlite3 import dbapi2 as sqlite
from sqlmodel import SQLModel
def save_locations():

    # Make sure the .data directory exists
    if not os.path.exists('.data'):
        os.mkdir('.data')

    # Remove the database file if it exists
    if os.path.exists('.data/locations.db'):
        os.remove('.data/locations.db')

    # Create the engine that connects to the SQLite database
    engine = create_engine('sqlite+pysqlite:///.data/locations.db', module=sqlite)

    # Define a session factory that will be used to interact with the database
    Session = sessionmaker(bind=engine)

    # Create the tables in the database
    SQLModel.metadata.create_all(engine)

    # Open a session and add the locations list to it
    with Session() as session:
        for location in locations:
            session.add(location)
        session.commit()

    # Close the database connection
    engine.dispose()
save_locations()

## Load Locations from sqlite

In [38]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlite3 import dbapi2 as sqlite
from holodeck import *
from sqlmodel import select
from sqlalchemy.orm import selectinload, joinedload

def get_saved_locations():
    engine = create_engine('sqlite+pysqlite:///.data/locations.db', module=sqlite)

    SessionLocal = sessionmaker(bind=engine)

    with SessionLocal() as session:
        locations = session.query(Location).options(
            joinedload(Location.buildings),
            joinedload(Location.ways),
            joinedload(Location.encounters).joinedload(Encounter.actions)
                .joinedload(Action.critter),
            joinedload(Location.encounters).joinedload(Encounter.actions)
                .joinedload(Action.character),
            joinedload(Location.encounters).joinedload(Encounter.actions)
                .joinedload(Action.item),
            joinedload(Location.encounters).joinedload(Encounter.actions)
                .joinedload(Action.building),
            joinedload(Location.encounters).joinedload(Encounter.triggers).joinedload(Trigger.way),
            joinedload(Location.encounters).joinedload(Encounter.triggers).joinedload(Trigger.building),
            joinedload(Location.characters)
        ).all()


        

        return [location for location in locations]

get_saved_locations()




# Remove the loaded objects from the session cache
# session.expunge_all()

# Close the database connection
# engine.dispose()

[Location(name='The City of Avarice', id=1, description='A towering metropolis with steampunk inspired buildings and ornate steam powered automobiles filling the air with soot. The city is filled with skyscrapers that reach up to the sky, interspersed with large clock towers and old cathedrals. The streets are cobblestone lined, and the buildings are all built of an obsidian-like substance.', encounters=[Encounter(probability=0.2, location_id=1, description='You hear a loud crash coming from the cathedral', id=2, actions=[Action(type='character', critter_id=None, item_id=None, id=2, encounter_id=2, character_id=2, building_id=None, building=None, character=Character(location_id=None, description='A robber, who has broken into the cathedral and is making off with a valuable artefact.', id=2, name='Robber'), critter=None, item=None)], triggers=[Trigger(type='building', building_id=3, id=2, way_id=None, encounter_id=2, way=None, building=Building(enterable=True, name='Cathedral of Avarice

## Generate Image Prompts

In [41]:
from holodeck.gpt_text import \
        generate_object_image_prompt, \
        generate_building_image_prompt, \
        generate_location_image_prompt

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

locations = get_saved_locations()
locations_image_prompts = []
objects_image_prompts = []
buildings_image_prompts = []

def generate_location_images(location):
    return [(location, generate_location_image_prompt(location))]

def generate_object_images(location):
    prompts = []
    for o in location.objects:
        prompts.append((o, generate_object_image_prompt(o, location)))
    return prompts

def generate_building_images(location):
    prompts = []
    for b in location.all_buildings:
        prompts.append((b, generate_building_image_prompt(b, location)))
    return prompts

with ThreadPoolExecutor() as executor:
    location_image_futures = [executor.submit(generate_location_images, location) for location in locations]
    object_image_futures = [executor.submit(generate_object_images, location) for location in locations]
    building_image_futures = [executor.submit(generate_building_images, location) for location in locations]

    for f in location_image_futures:
        for prompt in f.result():
            locations_image_prompts.append(prompt)
    for f in object_image_futures:
        for prompt in f.result():
            objects_image_prompts.append(prompt)
    for f in building_image_futures:
        for prompt in f.result():
            buildings_image_prompts.append(prompt)


img_prompts = locations_image_prompts + objects_image_prompts + buildings_image_prompts

obj_number = 0
for location in locations:
    obj_number += 1
    obj_number += len(location.all_buildings)
    obj_number += len(location.objects)


assert len(img_prompts) == obj_number, "not all descriptions got generated!"



<
{'prompt': '(Bird eye view:1.9)of a (metropolis:1.8) filled with (steampunk inspired buildings:1.3), (ornate steam powered automobiles:1.5) emitting (smoke:1.4), (towering skyscrapers:1.5) reaching up to the sky and (large clock towers:1.3), (old cathedrals:1.3) and (cobblestone lined streets:1.5). There is also a (Cathedral of Avarice:1.5) with grand stained glass windows and a large, ornate altar, a (tall clock tower:1.4) with intricate carvings and an array of gears and cogs and an (obsidian building:1.4) with a large door at its base. Everything is built of an (obsidian-like:1.3) substance. detailed, intricate, mesmerizing, beautiful, high resolution, realistic.', 'negative_prompt': 'Scribbles,Low quality,Low rated,Mediocre,3D rendering,Screenshot,Software,UI,((watermark)),(text),(overlay),getty images,(cropped),low'}

<
{'prompt': '(Long shot) of a (grand:1.8) (cathedral:1.8) with (ornate stained glass:1.6) windows and a (large:1.4) (ornate altar), located in a (towering:1.8) (m

## Generate Images

In [ ]:
from holodeck.gpt_image import generate_image
from IPython.display import display, Markdown
import PIL.Image as Image
import random


random.shuffle(img_prompts)

# img_prompts = img_prompts[:9]

images = []

for obj, gen_options in tqdm(img_prompts, desc="Images"):
    prompt = gen_options['prompt']
    negative_prompt = gen_options['negative_prompt']
    image_bytes = await generate_image(prompt=prompt, negative_prompt=negative_prompt)
    image = Image.open(image_bytes)
    display(Markdown(f"### {obj.name}"))
    display(Markdown(f"""
- {prompt}
  - *negative: {negative_prompt}*
    """))
    display(image)
    image_file_name = f".images/{obj.name}.png"
    image.save(image_file_name)
    image.close()
    image_bytes.close()
    del image_bytes
    images.append((obj, prompt, image_file_name))

len(images)


## Display Images

In [ ]:
import textwrap
import matplotlib.pyplot as plt
from matplotlib.patheffects import withStroke
from PIL.Image import Resampling

text_width = 30
dpi = 600.0
fig_width = 3.234

columns = 4
pic_width = fig_width * 0.27


def plot_images(images):

    # Estimate the number of rows needed based on the number of images
    num_rows = (len(images) + columns - 1) // columns
    fig_height = 0
    # Get the height of the first image adjusted for scale
    with Image.open(images[0][2]) as img:
        w, h = img.size
        aspect_ratio = float(w) / h
        pic_height = pic_width / aspect_ratio

        fig_height = pic_height * num_rows * 1.6

    fig = plt.figure(figsize=(fig_width, fig_height), dpi=dpi)

    # Define the path effect for the outline
    outline_effect = withStroke(linewidth=0.3, foreground='black')

    # Loop over the images and create a subplot for each
    for i, (obj, prompt, image_file_name) in enumerate(images):
        with Image.open(image_file_name) as image:
        
            # Resize the image
            w, h = image.size
            aspect_ratio = float(w) / h
            new_width = int(pic_width * dpi)
            new_height = int(new_width / aspect_ratio)
            with image.resize((new_width, new_height), Resampling.LANCZOS) as image_resized:
                # Create a subplot for the image
                ax = fig.add_subplot(len(images) // columns + 1, columns, i + 1)
                
                # Display the image
                ax.imshow(image_resized)

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal') # set aspect ratio to 1:1
        
        # Set the title to the obj.name
        # ax.set_title(obj.name, fontsize=4, color='magenta', pad=-10)
        
        # Wrap the prompt text to the desired width
        wrapped_prompt = textwrap.fill(prompt, width=text_width)
        wrapped_title = textwrap.fill(obj.name, width=14)
        
        # Display the wrapped prompt text below the title with black outline
        ax.text(0.08, -0.38, wrapped_prompt, ha='left', va='bottom', transform=ax.transAxes, fontsize=1.7, family='monospace', color='white', path_effects=[outline_effect])
        
        ax.text(1.0-0.08, -0.4, wrapped_title, ha='right', va='top', transform=ax.transAxes, fontsize=2.4, family='monospace', color='magenta', path_effects=[outline_effect])

    plt.subplots_adjust(wspace=0.03)

    fig.patch.set_facecolor('none')


plot_images(images)
